## Create corpus for entire Twitch Chat from user created clips

In [31]:
import new_token
import pandas as pd
import numpy as np
import requests
import json
import glob
import subprocess
from subprocess import *

In [7]:
t = new_token.token_class()
client_id, client_secret = t.get_info()
access_credentials = t.get_access_credentials()

In [ ]:
json.loads(r.content)['data']

In [13]:
url = 'https://api.twitch.tv/helix/streams?language=en&first=100'
r = requests.get(url, headers = access_credentials)

streamers = json.loads(r.content)['data']
master_data = []
for i in range(len(streamers)):
    data = []
    for key in streamers[i].keys():
        data.append(streamers[i][key])
    master_data.append(data)
        
streamers_df = pd.DataFrame(data = master_data, columns = streamers[i].keys())
streamers_df.head()

,id,user_id,user_login,user_name,game_id,game_name,type,title,viewer_count,started_at,language,thumbnail_url,tag_ids,tags,is_mature
0,43947677611,267160288,caseoh_,caseoh_,23020,ROBLOX,live,🟨LOSING MY MIND🟨 ESCAPE CASEOH FORNITE 🟨 MAP C...,69257,2024-04-02T01:27:39Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[],[English],False
1,43947131867,45098797,cdawgva,CDawgVA,509658,Just Chatting,live,CYCLETHON DAY 2 Hofu to Iwakuni | !idf !fundra...,23923,2024-04-02T00:09:20Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[],"[Japan, English]",False
2,42472077561,94753024,mizkif,Mizkif,509658,Just Chatting,live,20V1 W/ @ALINITY & @FANDY 🚨 SPONSORED BY GAMER...,19250,2024-04-01T17:47:14Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[],"[ADHD, Osteoporosis, OTK, STARFORGE, STEROIDS,...",False
3,43945700907,37402112,shroud,shroud,516575,VALORANT,live,I AM G303,15995,2024-04-01T20:41:15Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[],[English],False
4,50783470637,1027196985,iloveximm,iloveximm,460630,Tom Clancy's Rainbow Six Siege,live,2v2 Wagers W/ Jynxzi | !DISCORD | !TWITTER | !SUB,13993,2024-04-02T00:45:08Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[],[English],False


### Method 1: Create general corpus from all top streams
### Method 2: Create corpus by game type

In [ ]:
streamer_ids = list(streamers_df['user_id'])
streamer_names = list(streamers_df['user_name'])
#streamer_ids.append('71092938') # xQc
clip_data = []
clip_urls = []
num_clips = 15
for i in range(len(streamer_ids)):
    streamer_id = streamer_ids[i]
    streamer_name = streamer_names[i]
    #print(streamer_id)
    
    past_brodcast_times = [] # contains vod published datetimes
    url = f'https://api.twitch.tv/helix/videos?user_id={streamer_id}&sort=time&type=archive&first=100'
    r = requests.get(url, headers = access_credentials)
    
    # add vod timestamps to list
    for elem in json.loads(r.content)['data']:
        past_brodcast_times.append(elem['created_at'])

    if len(past_brodcast_times) == 0:
        continue
    # grab newest time stamp out of all the vods
    newest_video = max(past_brodcast_times) 
    

    url = f'https://api.twitch.tv/helix/clips?broadcaster_id={streamer_id}&first={num_clips}&started_at={newest_video}' 
    r = requests.get(url, headers = access_credentials)
    
    # grab top 15 clips 
    for elem in json.loads(r.content)['data']:
        #print(elem['url'])
        clip_urls.append(elem['url'])
    

    clip_data.append([streamer_name, streamer_id, len(past_brodcast_times)])
    
# download clips' chats
for url in clip_urls:
    temp_url = url.replace('https://clips.twitch.tv/','') # to make the path clean
    clip_chat = f'/Users/peter/Twitch/Twitch_VODs_Transcript/Clips/{temp_url}.json'
    
    try:
        cmd = ['chat_downloader',
         url,
         '--output',
         clip_chat]

        run(cmd)
    except:
        print('chat not available')   

In [34]:
def clean_author_id(x):
    try:
        return x['id']
    except:
        return np.NaN
     
        
master_chat = pd.DataFrame()
# read json file, clean it, and add it to single df
for clip_url in glob.glob('/Users/peter/Twitch/Twitch_VODs_Transcript/Clips/*'):
    chat = pd.read_json(clip_url)
    chat['author_id'] = chat['author'].apply(lambda x: clean_author_id(x))
    chat = chat.drop_duplicates(subset = ['author_id','message']) # drop spam messages from same user
    master_chat = pd.concat([master_chat,chat])


master_chat.reset_index(drop = True, inplace = True)

print(master_chat.shape)
master_chat.head()

(145811, 9)


,author,message,message_id,message_type,time_in_seconds,time_text,timestamp,emotes,author_id
0,"{'display_name': 'shovel_knight25', 'id': '422...",another thomas CLASSIC,3d7464a2-0d27-4f91-a65f-15c3d93e6928,text_message,0,0:00,2024-03-05 04:17:06.980,NaN,422869951
1,"{'badges': [{'clickAction': 'GET_TURBO', 'clic...",Enough pls,f863c502-7611-41c8-94d3-7f1381dd4f1c,text_message,1,0:01,2024-03-05 04:17:07.862,NaN,35079338
2,"{'badges': [{'clickAction': 'VISIT_URL', 'clic...",lmfao,7c12dc74-9994-44aa-b944-4b2d2a13a8dd,text_message,1,0:01,2024-03-05 04:17:08.698,NaN,668472199
3,"{'colour': '#0000FF', 'display_name': 'marlinb...",INCIDENT chance black,d891d979-c0e5-42a3-9931-6334dabe3622,text_message,2,0:02,2024-03-05 04:17:09.090,NaN,173134572
4,"{'badges': [{'clickAction': 'VISIT_URL', 'clic...",xdd Thomas,fade0f8a-1e4c-4982-a5ad-6c32cba7a6e7,text_message,2,0:02,2024-03-05 04:17:09.235,NaN,60749052


In [42]:
# grab all occurrences of messages that occur more than 5 times in the clips
counts = master_chat['message'].value_counts() 
valids = counts[counts>=5].index
keyword_freq = master_chat[master_chat['message'].isin(valids)]['message'].value_counts(
    normalize = True).reset_index() #.rename(columns = {'message':'freq','index':'message'})
keyword_freq

,message,proportion
0,KEKW,0.059723
1,LUL,0.025223
2,OMEGALUL,0.024829
3,HUH,0.019827
4,om,0.018523
...,...,...
2138,DinkDonk,0.000076
2139,Gladge,0.000076
2140,Guys?,0.000076
2141,PETTHEMODS,0.000076


In [43]:
keyword_freq.to_csv('corpus.csv',index = False)